In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.layers import Conv2D
from keras.layers import Add
from keras.models import Model

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
file_path_1 = "/content/drive/My Drive/IMDDataSample1deg_1981_2021.npy"
file_path_2 = "/content/drive/My Drive/BiasDataSample1deg_1981_2021.npy"
data_1 = np.load(file_path_1)
data_2 = np.load(file_path_2)

In [4]:
data_1.shape

(2132, 36, 36)

In [5]:
data_2.shape

(2132, 36, 36)

In [6]:
data_1

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [7]:
data_2

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [8]:
high_res_images = data_1[:1716, :, :]
low_res_images=data_2[:1716,:,:]

In [9]:
scaler = StandardScaler()
low_res_images = scaler.fit_transform(low_res_images.reshape(-1, 1)).reshape(low_res_images.shape)
high_res_images = scaler.transform(high_res_images.reshape(-1, 1)).reshape(high_res_images.shape)


In [10]:
X_train, X_temp, y_train, y_temp = train_test_split(low_res_images, high_res_images, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [11]:
# Define the U-Net architecture
def unet(input_shape):
    inputs = keras.layers.Input(input_shape)

    # Encoder
    conv1 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    # Bottleneck
    conv3 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)

    # Decoder
    up4 = keras.layers.UpSampling2D(size=(2, 2))(conv3)
    merge4 = keras.layers.concatenate([conv2, up4], axis=-1)
    conv4 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(merge4)
    conv4 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv4)

    up5 = keras.layers.UpSampling2D(size=(2, 2))(conv4)
    merge5 = keras.layers.concatenate([conv1, up5], axis=-1)
    conv5 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(merge5)
    conv5 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv5)

    outputs = keras.layers.Conv2D(1, (1, 1), activation='linear')(conv5)

    return keras.models.Model(inputs=[inputs], outputs=[outputs])


In [12]:
input_shape = (36, 36, 1)
model = unet(input_shape)


In [13]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])


In [14]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint = keras.callbacks.ModelCheckpoint('unet_model.h5', save_best_only=True)


In [15]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                    epochs=100, batch_size=32, callbacks=[early_stopping, model_checkpoint])


Epoch 1/100
38/38 [==============================] - 18s 62ms/step - loss: 1.1754 - mean_absolute_error: 0.6282 - val_loss: 0.3658 - val_mean_absolute_error: 0.3285
Epoch 2/100
38/38 [==============================] - 1s 32ms/step - loss: 0.2549 - mean_absolute_error: 0.2765 - val_loss: 0.1737 - val_mean_absolute_error: 0.2216
Epoch 3/100
38/38 [==============================] - 1s 29ms/step - loss: 0.1476 - mean_absolute_error: 0.2009 - val_loss: 0.1100 - val_mean_absolute_error: 0.1725
Epoch 4/100
38/38 [==============================] - 1s 32ms/step - loss: 0.1028 - mean_absolute_error: 0.1611 - val_loss: 0.0852 - val_mean_absolute_error: 0.1426
Epoch 5/100
38/38 [==============================] - 1s 31ms/step - loss: 0.0848 - mean_absolute_error: 0.1424 - val_loss: 0.0758 - val_mean_absolute_error: 0.1372
Epoch 6/100
38/38 [==============================] - 1s 29ms/step - loss: 0.0734 - mean_absolute_error: 0.1294 - val_loss: 0.0673 - val_mean_absolute_error: 0.1214
Epoch 7/100
38/

In [16]:
best_model = keras.models.load_model('unet_model.h5')


In [17]:
test_loss, test_mae = best_model.evaluate(X_test, y_test)

9/9 [==============================] - 1s 47ms/step - loss: 0.0573 - mean_absolute_error: 0.1004


In [18]:
bias_corrected_data = best_model.predict(low_res_images)

54/54 [==============================] - 1s 13ms/step


In [19]:
bias_corrected_data = bias_corrected_data.reshape(-1, 36, 36)

In [20]:
GCM_data_real=high_res_images.reshape(-1,36,36)

In [21]:
differences=bias_corrected_data-GCM_data_real

In [22]:
mean_differences=np.mean(differences,axis=(1,2))
global_mean_difference=np.mean(differences)
print(global_mean_difference)

-0.0013511245
